In [1]:
import ipynb.fs.full.dict_maker as dictionary
import requests
import feedparser
import pandas 

In [2]:
class livemint_reader:
    
    @staticmethod
    def read_feed(url:str) -> pandas.DataFrame:
        try:
            feed = feedparser.parse(url)
            titles = []
            links = []
            link_ids = []
            article_dates = []
            article_summaries = []
        
            for entry in feed.entries:
                titles.append(entry.title)
                links.append(entry.link)
                link_ids.append(entry.link[-19:][:14])
                article_dates.append(entry.published)
                article_summaries.append(entry.summary)

            df = pandas.DataFrame(
                    data={
                        'title': titles,
                        'link': links,
                        'link_id': link_ids,
                        'article_date': article_dates,
                        'article_summary': article_summaries
                    }
                )
            return df
        
        except feedparser.ExpatError as e:
            print(f"XML parsing error: {e}")
        except feedparser.NotFeedException as e:
            print(f"Not a valid feed: {e}")
        except feedparser.CharacterEncodingUnknown as e:
            print(f"Unknown character encoding error: {e}")
        except feedparser.ParseError as e:
            print(f"Parse error: {e}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

In [3]:
# One line tester
#url = dictionary.dict_to_json.get_value_from_file('Markets')
#df = livemint_reader.read_feed(url)
#print(df)

                                                title  \
0   NLC India Q4 Results: Net profit jumps 311% Yo...   
1   Upcoming IPOs: Two Coal India subsidiaries set...   
2   S&amp;P 500, Nasdaq futures drop over 1.3% ahe...   
3   Navratna PSU CONCOR declares date to consider ...   
4   Sensex, Nifty 50 fall for second day in a row—...   
5   Hyundai Motor looks to accelerate speed with E...   
6   SEBI threatens two offshore funds holding Adan...   
7   Stock market today: Sensex today tumbles 270 p...   
8   5600% rally in two years! Multibagger small-ca...   
9   BSE SME stock Rajesh Power Services jumps 20% ...   
10  Zerodha CEO Nithin Kamath unveils ‘much cooler...   
11  Stock market today: Nifty likely to move towar...   
12  The market is punishing Piramal Pharma. Is it ...   
13  FPIs pump  ₹18,620 crore into Indian equities ...   
14  Stock to buy: Axis Securities recommends Pitti...   
15  Navratna PSU NBCC shares rise 4% on multiple b...   
16     Emami’s growth plans set